In [822]:
import re
import smtplib
from datetime import datetime
import requests
from bs4 import BeautifulSoup as bs
import os

In [823]:
# Get unique values
def unique(xlist):
    unique_list = []
    
    for x in xlist:
        if x not in unique_list:
            unique_list.append(x)
    
    return unique_list

In [824]:
def search(list, keyword):
    for i in range(len(list)):
        if list[i] == keyword:
            return True
    return False

In [825]:
query_url = "https://www.xe.gr/property/results?transaction_name=rent&item_type=re_residence&maximum_price=960&minimum_size=57&geo_lat_from=38.0999672763109&geo_lng_from=24.061083822481493&geo_lat_to=37.792875685371634&geo_lng_to=23.530055641375128&sorting=update_desc"
crafted_urls = []
uniq_urls = []
start_page = 0  # Should be 1 at minimum
max_page_num = 3  # upper limit for the web page number to scrapp
links = []

for i in range(start_page, max_page_num):
    crafted_urls.append( query_url + "&page=" + str(i+1) )
    soup = bs(requests.get(crafted_urls[i]).content, 'lxml')
    
    for a_tag in soup.select('a[href*="enoikiaseis-katoikion"]'):
        links.append(a_tag['href'])
        
    links = links[:-15]


In [826]:
len(links)

204

In [827]:
uniq_urls = unique(links)

In [828]:
len(uniq_urls)

102

In [829]:
blacklist_areas = [
    "nosokomeio-paidwn", 
    "athhna-plateia-amerikhs",
    "vyrwnas",
    "zwgrafoy-kentro",
    "nosokomeio-paidwn",
    "athhna-koyntoyriwtika",
    "athhna-kynosargoys",
    "galatsi",
    "porto-rafth",
    "pagkrati",
    "ippokratoys",
    "ilion",
    "liosia",
    "keratsini"
    ]

In [830]:
blacklist_areas = unique(blacklist_areas)
blacklist_areas

['nosokomeio-paidwn',
 'athhna-plateia-amerikhs',
 'vyrwnas',
 'zwgrafoy-kentro',
 'athhna-koyntoyriwtika',
 'athhna-kynosargoys',
 'galatsi',
 'porto-rafth',
 'pagkrati',
 'ippokratoys',
 'ilion',
 'liosia',
 'keratsini']

In [831]:
def filter_blacklisted_areas(urls, blacklist_areas):
    # Filters based on blacklisted areas
        
    for url in urls:
        for area in blacklist_areas:
            if area in url:
                urls.remove(url)
                break # if blacklisted break the loop
    return urls

In [794]:
cleaned_urls= filter_blacklisted_areas(uniq_urls, blacklist_areas)

In [832]:
print(len(cleaned_urls))

95


In [833]:
class Home:
    def __init__(self, property_url):
        self.property_url = property_url
        
    def set_property_id(self):
        self.property_id = self.property_url.split('/')[6]

In [834]:
def make_homes(urls):
    Homes = []

    for i in range(0, len(urls)):
        Homes.append(Home(urls[i]))
        Homes[i].set_property_id()
    
    return Homes

In [836]:
Homes = make_homes(cleaned_urls)
len(Homes)

95

In [887]:
print(Homes[3].property_id)
print(Homes[3].property_url)

783198394
https://www.xe.gr/property/d/enoikiaseis-katoikion/783198394/hrakleio-prasinos-lofos-830-78?first_in_widget_ad_id=777324378&rank_in_widget=4&widget_name=search+results+list


In [888]:
h1 = Homes[3]

In [889]:
url = h1.property_url
url
# creating requests object
html = requests.get(url).content
  
# creating soup object
soup_html = bs(html, 'html.parser')
soup_lxml = bs(html, 'lxml')

In [890]:
# finding parent <ul> tag
parent = soup_html.find("body").find("ul")

# finding all <li> tags
text = list(parent.descendants)
#text
# printing the content in <li> tag
#print(text)
#for i in range(2, len(text), 2):
#    print(text[i], end=" ")
#for i in range(6, len(text), 2):
#    print(text[i], end=" ")


In [891]:
bedroom = int( text[text.index("Υπνοδωμάτια:") + 2].text )
bedroom

2

In [892]:
bathroom="NA"
if search(text, "Μπάνια:"):
    bathroom = int( text[text.index("Μπάνια:") + 2].text )
bathroom

1

In [893]:
import re
floor = int( re.findall(r'\d+', text[text.index("Όροφος:") + 2].text)[0] )
floor

3

In [894]:
house_construction = int(text[text.index("Έτος κατασκευής:") + 2].text)
house_construction

2008

In [895]:
house_reconstruction="NA"
if search(text, "Έτος ανακαίνισης:"):
    house_reconstruction = int(text[text.index("Έτος ανακαίνισης:") + 2].text)
house_reconstruction

'NA'

In [896]:
building_space = text[text.index("Κατάσταση:") + 2].text
building_space

'Καλή'

In [897]:
compass="NA"
if search(text, "Προσανατολισμός:"):
    compass = int(text[text.index("Προσανατολισμός:") + 2].text)
compass

ValueError: invalid literal for int() with base 10: 'Διαμπερές'

In [898]:
door = search(text, "Πόρτα ασφαλείας")
door

True

In [899]:
canopy = search(text, "Τέντες")
canopy

True

In [901]:
heating="NA"
if search(text, "Θέρμανση:"):
    heating = text[text.index("Θέρμανση:") + 2].text
heating

'Αυτόνομη θέρμανση με κεντρική εγκατάσταση'

In [902]:
title = soup_lxml.title.string
title

'\n  Διαμέρισμα προς ενοικίαση 78 τ.μ. Ηράκλειο (Πράσινος Λόφος) (Ιδιώτης) | xe.gr\n'

In [903]:
description = soup_lxml.p.string
description

'\n                    Πλησίον προαστιακού σταθμού και Αττικής οδού, σε άριστη κατάσταση\n                  '

In [904]:
price = int (re.findall(r'\d+', soup_lxml.find(class_='price').h2.text)[0])
price

830